In [67]:
%%capture
%run number.ipynb

# Memory

The memory is basically a long list of bytes. It start of empty. Everytime we need to make that list bigger we have to pay a gas fee.

Each memory cell can hold 1 byte (256 bits). A number between 0x00 (0) and 0xFF (255).

![title](static/memory.png)

In [68]:
class SimpleMemory:
    def __init__(self): self.memory = []
        
    def access(self, offset, size): return self.memory[offset:offset+size]
    def load  (self, offset):       return self.access(offset, 32)
    def store (self, offset, value): self.memory[offset:len(value)] = value

We need to add logic in order to calculate the exact gas of extending the memory.

I don't think we need to understand every detail here. Important to know is this: Accessing new offsets increases the memory. Memory expansion costs gas non-linearly. Making it more costly to access higher offsets.

In [69]:
def calc_memory_expansion_gas(memory_byte_size):
    memory_size_word = (memory_byte_size + 31) / 32
    memory_cost = (memory_size_word ** 2) / 512 + (3 * memory_size_word)
    return round(memory_cost)

Lets add the gas expansion calculation to the Memory

In [70]:
class Memory(SimpleMemory):
    def store(self, offset, value):
        memory_expansion_cost = 0
        
        if len(self.memory) <= offset:
            
            expansion_size = 0
            
            # initialize memory with 32 zeros if it is empty
            if len(self.memory) == 0:
                expansion_size = 32
                self.memory = [0x00 for _ in range(32)]
                
            # extend memory if needed
            if len(self.memory) - offset < 0:
                expansion_size += offset - len(self.memory)
                self.memory.append(0x00 * expansion_size)
                
            memory_expansion_cost = calc_memory_expansion_gas(expansion_size)
                
        super().store(offset, value)
        return memory_expansion_cost

In [81]:
memory = Memory()

In [88]:
number = Number([0x12, 0x33])

In [92]:
number.bytes

[18, 51]

In [95]:
memory.store(0, number.bytes)
memory.load(0)

[18,
 51,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]